In [183]:
import geopandas as gpd
import pandas as pd 
import os

In [184]:
from gpd_loader import get_gpd

dir_path = '용도별건물공간확보_부산'
자치구_list, df_dict = get_gpd(dir_path)

In [185]:
df_dict.keys()

dict_keys(['강서구', '금정구', '기장군', '남구', '동구', '동래구', '부산진구', '북구', '사상구', '사하구', '서구', '수영구', '연제구', '영도구', '중구', '해운대구'])

In [186]:
df_dict_copy = df_dict.copy()

In [ ]:
columns = ['A25' , 'A27', 'A29']#25:주요용도명, 27:세부용도명, 29:건물용도명

In [188]:
df_dict_copy['강서구'].A29.unique()

array(['농수산용', '기타', '상업용', '주거용', '문교사회용', '공업용', None, '공공용'],
      dtype=object)

In [189]:
def fill_na(df):
    df = df.dropna(subset=columns, how='all')
    df.loc[df['A27'].isna(), 'A27'] = df.loc[df['A27'].isna(), 'A25']
    df.loc[df['A27'].isna(), 'A27'] = df.loc[df['A27'].isna(), 'A29']
    return df

In [190]:
for df_key, df in df_dict_copy.items():
    df_dict_copy[df_key] = fill_na(df)
    df_dict_copy[df_key] = df_dict_copy[df_key][columns]
    

In [191]:
a27_list = list(set(val for df in df_dict_copy.values() for val in df['A27'].unique()))

In [192]:
with open('a27_list.txt', 'w', encoding='utf-8') as f:
    for item in a27_list:
        f.write(f"{item}\n")

In [193]:
len(a27_list)

352

In [194]:
with open('nearby.txt', 'r', encoding='utf-8') as f:
    nearby_list = [line.strip() for line in f.readlines()]

In [195]:
len(nearby_list)

19

In [196]:
for key in df_dict.keys():
    df_dict[key] = fill_na(df_dict[key])
    df_dict[key] = df_dict[key][df_dict[key]['A27'].isin(nearby_list)]
    print(f'{key}의 A27 값 {len(df_dict[key].A27.unique())} \n: ',df_dict[key].A27.unique())

강서구의 A27 값 12 
:  ['단독주택' '학원' '다가구주택' '영유아보육시설' '유치원' '초등학교' '다중주택' '주거용' '어린이집' '중학교'
 '다세대주택' '아파트']
금정구의 A27 값 17 
:  ['단독주택' '다가구주택' '학원' '기타아동관련시설' '아동복지시설' '초등학교' '다중주택' '어린이집' '영유아보육시설'
 '유치원' '중학교' '지역아동센터' '다세대주택' '노유자시설' '공동주택' '주거용' '아파트']
기장군의 A27 값 16 
:  ['단독주택' '다가구주택' '학원' '기타아동관련시설' '영유아보육시설' '초등학교' '아동복지시설' '공동주택' '어린이집'
 '지역아동센터' '유치원' '중학교' '다중주택' '다세대주택' '노유자시설' '아파트']
남구의 A27 값 15 
:  ['다가구주택' '단독주택' '학원' '공동주택' '초등학교' '유치원' '중학교' '영유아보육시설' '어린이집' '기타아동관련시설'
 '노유자시설' '다중주택' '아동복지시설' '다세대주택' '아파트']
동구의 A27 값 16 
:  ['단독주택' '학원' '다가구주택' '지역아동센터' '유치원' '초등학교' '중학교' '영유아보육시설' '다중주택' '공동주택'
 '어린이집' '기타아동관련시설' '아동관련시설' '다세대주택' '노유자시설' '아파트']
동래구의 A27 값 16 
:  ['단독주택' '다가구주택' '기타아동관련시설' '어린이집' '초등학교' '중학교' '학원' '영유아보육시설' '유치원' '공동주택'
 '노유자시설' '지역아동센터' '다중주택' '다세대주택' '아동복지시설' '아파트']
부산진구의 A27 값 17 
:  ['단독주택' '다가구주택' '학원' '초등학교' '중학교' '유치원' '기타아동관련시설' '어린이집' '아동복지시설'
 '영유아보육시설' '노유자시설' '다세대주택' '아동관련시설' '공동주택' '지역아동센터' '다중주택' '아파트']
북구의 A27 값 15 
:  ['학원' '단독주택' '다가구주택' '

In [197]:
for key in df_dict.keys():
    print(len(df_dict[key]))

9589
15922
8864
13782
9862
12808
17549
8220
9316
16436
9608
10417
9889
10325
3750
13921


In [198]:
os.makedirs('용도별건물공간확보_부산_가공', exist_ok=True)
for name in df_dict.keys():
    os.makedirs(f'용도별건물공간확보_부산_가공/{name}', exist_ok=True)
    df_dict[name].to_file(f'용도별건물공간확보_부산_가공/{name}/{name}.shp', encoding='cp949')